<a href="https://colab.research.google.com/github/DianaBravoPerez/EDP-1/blob/main/GS_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Método de **Gauss–Seidel** ·

Buscamos $u$ tal que $\nabla^2u=0$ con
$$u(0,y)=0,\quad u(2,y)=y(2-y),\quad u(x,0)=0,\quad u(x,2)=\begin{cases}x,&0<x<1,\\2-x,&1\le x<2.\end{cases}$$
Se toma $h=\tfrac12$ y se usa el esquema de 5 puntos. En los 9 nodos interiores escribimos la forma fija
$$x^{k+1}=T\,x^{k}+c,$$
y actualizamos con **Gauss–Seidel**

In [2]:
import numpy as np
import pandas as pd
from IPython.display import display
pd.set_option('display.precision', 6)

In [3]:
# Matriz T (1/4 en vecinos) con el orden indicado arriba
T = np.array([
    [0,   1/4, 0,   1/4, 0,   0,   0,   0,   0  ],
    [1/4, 0,   1/4, 0,   1/4, 0,   0,   0,   0  ],
    [0,   1/4, 0,   0,   0,   1/4, 0,   0,   0  ],
    [1/4, 0,   0,   0,   1/4, 0,   1/4, 0,   0  ],
    [0,   1/4, 0,   1/4, 0,   1/4, 0,   1/4, 0  ],
    [0,   0,   1/4, 0,   1/4, 0,   0,   0,   1/4],
    [0,   0,   0,   1/4, 0,   0,   0,   1/4, 0  ],
    [0,   0,   0,   0,   1/4, 0,   1/4, 0,   1/4],
    [0,   0,   0,   0,   0,   1/4, 0,   1/4, 0  ]
], dtype=float)

# Vector c con las contribuciones de frontera
c = np.array([0, 0, 3/4, 0, 0, 1, 1/2, 1, 5/4], dtype=float)

# Punto inicial: distinto al clásico; aquí arrancamos en cero
x0 = np.zeros(9)

display(pd.DataFrame(T))
display(pd.DataFrame(c, columns=["c"]))

,0,1,2,3,4,5,6,7,8
0,0.00,0.25,0.00,0.25,0.00,0.00,0.00,0.00,0.00
1,0.25,0.00,0.25,0.00,0.25,0.00,0.00,0.00,0.00
2,0.00,0.25,0.00,0.00,0.00,0.25,0.00,0.00,0.00
3,0.25,0.00,0.00,0.00,0.25,0.00,0.25,0.00,0.00
4,0.00,0.25,0.00,0.25,0.00,0.25,0.00,0.25,0.00
5,0.00,0.00,0.25,0.00,0.25,0.00,0.00,0.00,0.25
6,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.25,0.00
7,0.00,0.00,0.00,0.00,0.25,0.00,0.25,0.00,0.25
8,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.25,0.00


,c
0,0.00
1,0.00
2,0.75
3,0.00
4,0.00
5,1.00
6,0.50
7,1.00
8,1.25


### Parámetros que ingresa el usuario

In [4]:
tol_in  = input("Ingresa la tolerancia (ej. 1e-4): ").strip().replace(',', '.')
nmax_in = input("Máximo de iteraciones (enter = 1000): ").strip()
TOL  = float(tol_in) if tol_in else 1e-4
TOL  = TOL if TOL > 0 else 1e-4
NMAX = int(nmax_in) if nmax_in else 1000
NMAX = NMAX if NMAX > 0 else 1000

Ingresa la tolerancia (ej. 1e-4): 0.0001
Máximo de iteraciones (enter = 1000): 10000


## Iteración **Gauss–Seidel** sobre $x = T x + c$

In [5]:
def gauss_seidel_fixed_point(T, c, x0, tol, nmax):
    n = len(x0)
    x = x0.copy()
    log = []
    for k in range(1, nmax+1):
        x_old = x.copy()
        # Barrido GS: usa valores ya actualizados en las posiciones previas
        for i in range(n):
            s_izq = T[i, :i] @ x[:i]      # usa x ya actualizado
            s_der = T[i, i+1:] @ x_old[i+1:]  # usa x viejo
            x[i] = s_izq + s_der + c[i]
        err = float(np.linalg.norm(x - x_old, ord=np.inf))
        log.append([k, *x.tolist(), err])
        if err < tol:
            return x, k, pd.DataFrame(log, columns=["Iteración", "x1","x2","x3","x4","x5","x6","x7","x8","x9","Epsilon"]), True
    return x, k, pd.DataFrame(log, columns=["Iteración", "x1","x2","x3","x4","x5","x6","x7","x8","x9","Epsilon"]), False

sol, iters, tabla, ok = gauss_seidel_fixed_point(T, c, x0, TOL, NMAX)
print(f"Convergió: {ok} en {iters} iteraciones (TOL={TOL})\n")
display(tabla)

Convergió: True en 16 iteraciones (TOL=0.0001)



,Iteración,x1,x2,x3,x4,x5,x6,x7,x8,x9,Epsilon
0,1,0.000000,0.000000,0.750000,0.000000,0.000000,1.187500,0.500000,1.125000,1.828125,1.828125
1,2,0.000000,0.187500,1.093750,0.125000,0.656250,1.894531,0.812500,1.824219,2.179688,0.707031
2,3,0.078125,0.457031,1.337891,0.386719,1.140625,2.164551,1.052734,2.093262,2.314453,0.484375
3,4,0.210938,0.672363,1.459229,0.601074,1.382812,2.289124,1.173584,2.217712,2.376709,0.242188
4,5,0.318359,0.790100,1.519806,0.718689,1.503906,2.350105,1.234100,2.278679,2.407196,0.121094
5,6,0.377197,0.850227,1.550083,0.778801,1.564453,2.380433,1.264370,2.309005,2.422359,0.060547
6,7,0.407257,0.880448,1.565220,0.809020,1.594727,2.395577,1.279506,2.324148,2.429931,0.030273
7,8,0.422367,0.895579,1.572789,0.824150,1.609863,2.403146,1.287075,2.331717,2.433716,0.015137
8,9,0.429932,0.903146,1.576573,0.831717,1.617432,2.406930,1.290859,2.335502,2.435608,0.007568
9,10,0.433716,0.906930,1.578465,0.835502,1.621216,2.408822,1.292751,2.337394,2.436554,0.003784


### Malla 3×3 (de arriba hacia abajo)

In [6]:
# Reorganiza (U11,U21,U31, U12,U22,U32, U13,U23,U33) a una grilla 3x3
U = sol.reshape(3,3, order='F')
grid = np.vstack([U[:,2], U[:,1], U[:,0]])  # y: alto→bajo
df_grid = pd.DataFrame(grid, index=["y=1.5","y=1.0","y=0.5"], columns=["x=0.5","x=1.0","x=1.5"])
display(df_grid)

,x=0.5,x=1.0,x=1.5
y=1.5,1.294613,2.339256,2.437485
y=1.0,0.839227,1.624941,2.410685
y=0.5,0.437441,0.910655,1.580328
